In [32]:
import json
import os
import random
import time

import bson
import consts
import pymongo
from dotenv import load_dotenv
from rio_requests import character_run_summaries, get_run_from_id, get_title_range
from schema import Character, Run, RunSummary
from tqdm import tqdm

load_dotenv()
DB_URL = os.getenv('DB_URL')
client = pymongo.MongoClient(DB_URL)

TITLE_SCORE = 3650

database_runs = open('data/raideriknow.runs.json', 'r')
database_runs = json.load(database_runs)
database_runs = [Run.from_json(run) for run in database_runs]
database_run_ids = { run.keystone_run_id: run for run in database_runs }

database_characters = open('data/raideriknow.characters.json', 'r')
database_characters = json.load(database_characters)
database_characters = [Character.from_json(character) for character in database_characters]
database_characters = sorted(database_characters, key=lambda character: character.name)
database_character_ids = { character.id: character for character in database_characters }
database_character_oids = { character._id: character for character in database_characters }


In [33]:
# TODO: i could compute all of the ranges (p90, etc.) pretty easily, should definitely consider
title_info = get_title_range()['cutoffs']['p999']['all']
title_score = title_info['quantileMinValue']
num_title_players = title_info['quantilePopulationCount']
print(f'Title score: {title_score} ({num_title_players} players)')

Title score: 3651.26 (1443 players)


In [41]:

"""
rough calculation
250,000 runs
each run has 5 characters
1,250,000 character-run operations
"""
def precompute_runs_per_character(d_runs) :
    runs_per_character = {}
    for run in tqdm(d_runs) :
        for character_oid in run.roster :
            character_id = str(character_oid)
            if character_id not in runs_per_character :
                runs_per_character[character_id] = []
            runs_per_character[character_id].append(run)

    return runs_per_character

runs_per_char = precompute_runs_per_character(database_runs)


def get_runs_with_character(character) :
    return runs_per_char[str(character._id)] if str(character._id) in runs_per_char else []

def get_runs_with_characters(characters) :
    runs = []
    for character in characters :
        runs += get_runs_with_character(character)
    return runs

# https://www.reddit.com/r/wow/comments/13vqsbw/an_accurate_formula_for_m_score_calculation_in/
def calc_run_score (run) :
    # we assume all runs are above 20, so have 3 affixes
    # this might need to change for future seasons
    # assume 0 timer bonus
    return 7 * run.mythic_level + 30

class CharTopRuns (object) :
    def __init__ (self, char) :
        self.char = char
        runs = get_runs_with_character(char)
        self.top_runs = {
            dungeon_id: {
                'Tyrannical': None,
                'Fortified': None,
            }
            for dungeon_id in consts.DUNGEON_IDS
        }

        for run in runs :
            dungeon_id = run.dungeon['id']
            major_affix = 'Fortified' if 'Fortified' in run.weekly_modifiers else 'Tyrannical'
            score = calc_run_score(run)

            if self.top_runs[dungeon_id][major_affix] is None or score > calc_run_score(self.top_runs[dungeon_id][major_affix]) :
                self.top_runs[dungeon_id][major_affix] = run
    

    def overall_score (self) :
        score = 0

        for dungeon_id in consts.DUNGEON_IDS :
            score_1 = 0
            score_2 = 0
            if self.top_runs[dungeon_id]['Fortified'] is not None :
                score_1 = calc_run_score(self.top_runs[dungeon_id]['Fortified'])
            
            if self.top_runs[dungeon_id]['Tyrannical'] is not None :
                score_2 = calc_run_score(self.top_runs[dungeon_id]['Tyrannical'])

            dung_score = 1.5 * max(score_1, score_2) + 0.5 * min(score_1, score_2)

            score += dung_score
            
        return score
    

    def top_runs (self) :
        return self.top_runs
    
    def top_runs_flattened (self) :
        return list(filter(lambda run: run, [run for dungeon in self.top_runs.values() for run in dungeon.values()]))
    

    def top_run_levels (self) :
        levels = {
            dungeon_id: {
                'Fortified': None,
                'Tyrannical': None,
            }
            for dungeon_id in consts.DUNGEON_IDS
        }
        for dungeon_id in consts.DUNGEON_IDS :
            levels[dungeon_id]['Fortified'] = self.top_runs[dungeon_id]['Fortified'].mythic_level if self.top_runs[dungeon_id]['Fortified'] else 0
            levels[dungeon_id]['Tyrannical'] = self.top_runs[dungeon_id]['Tyrannical'].mythic_level if self.top_runs[dungeon_id]['Tyrannical'] else 0   
             
        return levels

ex_char_name = "Vexea"
example_character = list(filter(lambda char: char.name == ex_char_name, database_characters))[0]
ex_char_top_runs = CharTopRuns(example_character)
print(f'Example character: {example_character.name} ({example_character.id})')
print(f'Overall score: {ex_char_top_runs.overall_score()}')
# not completely accurate, but the data was taken a while ago



def get_title_range_chars (title_score) :
    title_range_chars = []
    for character in tqdm(database_characters) :
        char_top_runs = CharTopRuns(character)
        if char_top_runs.overall_score() >= title_score :
            title_range_chars.append(character)
    return title_range_chars


def get_title_top_runs (title_range_chars) :
    title_top_runs = []
    for character in tqdm(title_range_chars) :
        char_top_runs = CharTopRuns(character)
        title_top_runs.append(char_top_runs)
    return title_top_runs

title_range_chars = get_title_range_chars(title_score)
title_top_runs = get_title_top_runs(title_range_chars)
print(f'Number of characters in title range: {len(title_range_chars)}')
print(f'Example character: {title_range_chars[0].name} ({title_range_chars[0].id})')
print("Example title top run id: " + str(title_top_runs[0].top_runs[9028]['Fortified'].to_json()['keystone_run_id']))
print(title_top_runs[0].top_run_levels())

100%|██████████| 260474/260474 [00:00<00:00, 281549.62it/s]


Example character: Vexea (135683693)
Overall score: 3742.0


100%|██████████| 1383/1383 [00:00<00:00, 4159.69it/s]

Number of characters in title range: 1383
Example character: Aaeldin (60145404)
Example title top run id: 25370700
{9028: {'Fortified': 30, 'Tyrannical': 30}, 7805: {'Fortified': 31, 'Tyrannical': 31}, 1000010: {'Fortified': 31, 'Tyrannical': 29}, 1000011: {'Fortified': 29, 'Tyrannical': 29}, 7673: {'Fortified': 30, 'Tyrannical': 29}, 7109: {'Fortified': 29, 'Tyrannical': 29}, 4738: {'Fortified': 29, 'Tyrannical': 29}, 9424: {'Fortified': 30, 'Tyrannical': 30}}


1383 characters seen in database, 1443 actual - about 5% error

In [42]:
def count_top_run_levels(title_top_runs) :
    level_counts = {
        dungeon_id: {
            'Fortified': {},
            'Tyrannical': {},
        }
        for dungeon_id in consts.DUNGEON_IDS
    }

    for char_top_runs in title_top_runs :
        levels = char_top_runs.top_run_levels()
        for dungeon_id in consts.DUNGEON_IDS :
            for affix in ['Fortified', 'Tyrannical'] :
                level = levels[dungeon_id][affix]
                if level not in level_counts[dungeon_id][affix] :
                    level_counts[dungeon_id][affix][level] = 0
                level_counts[dungeon_id][affix][level] += 1

    return level_counts

level_counts = count_top_run_levels(title_top_runs)
print(level_counts)

{9028: {'Fortified': {30: 559, 29: 648, 31: 111, 28: 44, 32: 19, 27: 2}, 'Tyrannical': {30: 361, 29: 796, 28: 148, 31: 56, 26: 1, 32: 15, 27: 5, 25: 1}}, 7805: {'Fortified': {31: 115, 30: 532, 29: 697, 28: 24, 32: 13, 27: 2}, 'Tyrannical': {31: 59, 29: 819, 30: 404, 28: 93, 32: 6, 27: 2}}, 1000010: {'Fortified': {31: 57, 29: 815, 30: 457, 28: 42, 32: 11, 0: 1}, 'Tyrannical': {29: 729, 28: 462, 30: 142, 26: 5, 31: 24, 27: 21}}, 1000011: {'Fortified': {29: 446, 28: 779, 26: 7, 30: 96, 27: 45, 31: 10}, 'Tyrannical': {29: 340, 28: 809, 27: 173, 30: 46, 31: 10, 26: 5}}, 7673: {'Fortified': {30: 177, 29: 827, 28: 320, 27: 16, 31: 41, 26: 2}, 'Tyrannical': {29: 465, 28: 786, 27: 72, 30: 58, 26: 1, 25: 1}}, 7109: {'Fortified': {29: 226, 28: 996, 26: 7, 27: 113, 30: 36, 25: 2, 24: 3}, 'Tyrannical': {29: 55, 28: 475, 27: 767, 26: 77, 0: 1, 25: 8}}, 4738: {'Fortified': {29: 176, 28: 902, 27: 281, 26: 12, 25: 1, 30: 10, 24: 1}, 'Tyrannical': {29: 196, 27: 181, 28: 968, 26: 8, 30: 27, 25: 3}}, 9424